### Extract Planning Areas of Singapore data from Wikipedia

In [1]:
import pandas as pd
url = 'https://en.wikipedia.org/wiki/Planning_Areas_of_Singapore'
dfs = pd.read_html(url)

In [2]:
#Find out how many tables were imported
len(dfs)

4

In [3]:
#Check the contents of Table 2
dfs[2].head()

,Name (English),Malay,Chinese,Pinyin,Tamil,Region,Area (km2),Population[6],Density (/km2)
0,Ang Mo Kio,NaN,宏茂桥,Hóng mào qiáo,ஆங் மோ கியோ,North-East,13.94,165710,12000
1,Bedok,*,勿洛,Wù luò,பிடோக்,East,21.69,281300,13000
2,Bishan,NaN,碧山,Bì shān,பீஷான்,Central,7.62,88490,12000
3,Boon Lay,NaN,文礼,Wén lǐ,பூன் லே,West,8.23,30,3.6
4,Bukit Batok,*,武吉巴督,Wǔjí bā dū,புக்கிட் பாத்தோக்,West,11.13,144410,13000


##### Table 2 contains the data needed

### Data Cleanup

In [4]:
# Assign Table 2 to a new dataframe
df=dfs[2]
df.head()


,Name (English),Malay,Chinese,Pinyin,Tamil,Region,Area (km2),Population[6],Density (/km2)
0,Ang Mo Kio,NaN,宏茂桥,Hóng mào qiáo,ஆங் மோ கியோ,North-East,13.94,165710,12000
1,Bedok,*,勿洛,Wù luò,பிடோக்,East,21.69,281300,13000
2,Bishan,NaN,碧山,Bì shān,பீஷான்,Central,7.62,88490,12000
3,Boon Lay,NaN,文礼,Wén lǐ,பூன் லே,West,8.23,30,3.6
4,Bukit Batok,*,武吉巴督,Wǔjí bā dū,புக்கிட் பாத்தோக்,West,11.13,144410,13000


In [5]:
# Drop unnecessary columns
df = df.drop(["Malay", "Chinese", "Pinyin",'Tamil'], axis=1)
# Rename columns
df.columns = ['Neighborhood', 'Region', 'Area(km2)', 'Population', 'Density(/km2)']
# Drop rows with '*' as Population
df = df.set_index("Population")
df = df.drop('*', axis=0)
df = df.reset_index()
# Assign Population as INT
df.Population = df.Population.astype(int)
# Drop 'North-Eastern Islands' row
df = df.set_index("Neighborhood")
df = df.drop('North-Eastern Islands', axis=0)
df = df.reset_index()
#Replace Neighborhood Names (Downtown Core to Bugis, Western Islands to Jurong Island, Museum to Dhoby Ghaut as the original names were not
#recognized by OpenSourceMaps and resulted in errors when getting the latitude and longitude of Neighborhood
df=df.replace({'Downtown Core': 'Bugis', 'Western Islands': 'Jurong Island','Museum':'Dhoby Ghaut'})
df

,Neighborhood,Population,Region,Area(km2),Density(/km2)
0,Ang Mo Kio,165710,North-East,13.94,12000
1,Bedok,281300,East,21.69,13000
2,Bishan,88490,Central,7.62,12000
3,Boon Lay,30,West,8.23,3.6
4,Bukit Batok,144410,West,11.13,13000
5,Bukit Merah,151870,Central,14.34,11000
6,Bukit Panjang,140820,West,8.99,16000
7,Bukit Timah,77280,Central,17.53,4400
8,Changi,2080,East,40.61,62.3
9,Choa Chu Kang,187510,West,6.11,31000


### Getting Latitude and Longitude of Neighborhoods

In [6]:
from geopy.geocoders import Nominatim

In [7]:
geolocator = Nominatim(user_agent="sg_explorer")

def get_latlng(address):
    address = address + ', Singapore'
    location = geolocator.geocode(address)
    lat = location.latitude
    lng = location.longitude
    #print(lat,lng)
    return lat, lng


addr = df['Neighborhood']

coords = [ get_latlng(i) for i in addr.tolist() ]

df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

df.head()


,Neighborhood,Population,Region,Area(km2),Density(/km2),Latitude,Longitude
0,Ang Mo Kio,165710,North-East,13.94,12000,1.369842,103.846609
1,Bedok,281300,East,21.69,13000,1.323976,103.930216
2,Bishan,88490,Central,7.62,12000,1.351452,103.848250
3,Boon Lay,30,West,8.23,3.6,1.345640,103.711802
4,Bukit Batok,144410,West,11.13,13000,1.349057,103.749591


In [8]:
address = 'Singapore, Singapore'

#geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Singapore are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Singapore are 1.3408528, 103.878446863736.


### Visualize the Neighborhoods and its Population

In [9]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [10]:
map_sg = folium.Map(location=[latitude, longitude], zoom_start=11)

for i in range(0,len(df)):
    #label = '{}'.format(neighborhood)
    #label = folium.Popup(label, parse_html=True)
    
    folium.Circle(
      location=[df.iloc[i]['Latitude'], df.iloc[i]['Longitude']],
      popup=(df.iloc[i]['Neighborhood']+'\n'+str(df.iloc[i]['Population'])),
      radius=df.iloc[i]['Population']/150,
      color='blue',
      fill=True,
      fill_color='#3186cc'
   ).add_to(map_sg)
map_sg

In [11]:
# Display the Neighborhood sorted by Population size in descending order
df.sort_values(by=['Population'],ascending = False).head(10)

,Neighborhood,Population,Region,Area(km2),Density(/km2),Latitude,Longitude
1,Bedok,281300,East,21.69,13000,1.323976,103.930216
15,Jurong West,266720,West,14.69,27000,1.339636,103.707339
39,Tampines,257110,East,20.89,12506.2,1.354653,103.943571
46,Woodlands,252530,North,13.59,18424,1.436897,103.786216
34,Sengkang,240640,North-East,10.59,19511,1.390949,103.895175
13,Hougang,223010,North-East,13.93,16000,1.373360,103.886091
47,Yishun,214940,North,21.24,9507.2,1.428136,103.833694
9,Choa Chu Kang,187510,West,6.11,31000,1.389260,103.743728
0,Ang Mo Kio,165710,North-East,13.94,12000,1.369842,103.846609
28,Punggol,161570,North-East,9.34,17000,1.398033,103.907331


### Using FourSquare to retrieve Venues in the Neighborhoods for further analysis

In [12]:
CLIENT_ID = 'K20PMJTQA0B0HSIODFBFV0TOA41RWEI5DQ4XOACEAX33AXAV' # your Foursquare ID
CLIENT_SECRET = 'GMVML3AYKFBP0AMSW0I3BVU3XZ0WNFKK2Q2WSSCIP3WADFTP' # your Foursquare Secret
VERSION = '20190612'

In [13]:
import requests # library to handle requests

LIMIT = 200 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
sg_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


Ang Mo Kio
Bedok
Bishan
Boon Lay
Bukit Batok
Bukit Merah
Bukit Panjang
Bukit Timah
Changi
Choa Chu Kang
Clementi
Bugis
Geylang
Hougang
Jurong East
Jurong West
Kallang
Lim Chu Kang
Mandai
Marine Parade
Dhoby Ghaut
Newton
Novena
Orchard
Outram
Pasir Ris
Paya Lebar
Pioneer
Punggol
Queenstown
River Valley
Rochor
Seletar
Sembawang
Sengkang
Serangoon
Singapore River
Southern Islands
Sungei Kadut
Tampines
Tanglin
Tengah
Toa Payoh
Tuas
Jurong Island
Western Water Catchment
Woodlands
Yishun


In [15]:
print(sg_venues.shape)
sg_venues.head()

(2991, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ang Mo Kio,1.369842,103.846609,Old Chang Kee,1.369094,103.848389,Snack Place
1,Ang Mo Kio,1.369842,103.846609,Bun Master,1.369242,103.849031,Bakery
2,Ang Mo Kio,1.369842,103.846609,Subway,1.369136,103.847612,Sandwich Place
3,Ang Mo Kio,1.369842,103.846609,MOS Burger,1.369170,103.847831,Burger Joint
4,Ang Mo Kio,1.369842,103.846609,PLAYe,1.369109,103.848225,Hobby Shop


In [16]:
sg_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ang Mo Kio,87,87,87,87,87,87
Bedok,97,97,97,97,97,97
Bishan,65,65,65,65,65,65
Boon Lay,74,74,74,74,74,74
Bugis,100,100,100,100,100,100
Bukit Batok,40,40,40,40,40,40
Bukit Merah,100,100,100,100,100,100
Bukit Panjang,45,45,45,45,45,45
Bukit Timah,26,26,26,26,26,26


In [17]:
print('There are {} uniques categories.'.format(len(sg_venues['Venue Category'].unique())))

There are 283 uniques categories.


In [18]:
# one hot encoding
sg_onehot = pd.get_dummies(sg_venues[['Venue Category']], prefix="", prefix_sep="")
sg_onehot.head()
# add neighborhood column back to dataframe
sg_onehot['Neighborhood'] = sg_venues['Neighborhood']
sg_onehot['Neighborhood']

# move neighborhood column to the first column
#fixed_columns = [sg_onehot.columns[-1]] + list(sg_onehot.columns[:-1])
#fixed_columns
sg_onehot = sg_onehot.set_index("Neighborhood")
sg_onehot = sg_onehot.reset_index()
#sg_onehot = sg_onehot[fixed_columns]
sg_onehot.head()


,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,...,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Ang Mo Kio,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Ang Mo Kio,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Ang Mo Kio,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Ang Mo Kio,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ang Mo Kio,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
sg_onehot.shape

(2991, 283)

In [20]:
sg_grouped = sg_onehot.groupby('Neighborhood').mean().reset_index()
sg_grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,...,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Ang Mo Kio,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000
1,Bedok,0.000000,0.000000,0.000000,0.000000,0.000000,0.010309,0.000000,0.00,0.00000,...,0.000000,0.00,0.000000,0.000000,0.00,0.010309,0.00,0.000000,0.00,0.000000
2,Bishan,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000
3,Boon Lay,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,...,0.000000,0.00,0.000000,0.000000,0.00,0.013514,0.00,0.000000,0.00,0.000000
4,Bugis,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.01000,...,0.000000,0.00,0.010000,0.010000,0.00,0.000000,0.01,0.010000,0.00,0.000000
5,Bukit Batok,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000
6,Bukit Merah,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.00000,...,0.000000,0.00,0.010000,0.000000,0.00,0.000000,0.00,0.010000,0.00,0.000000
7,Bukit Panjang,0.000000,0.000000,0.000000,0.000000,0.000000,0.022222,0.000000,0.00,0.00000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.022222,0.00,0.000000
8,Bukit Timah,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000
9,Changi,0.017857,0.053571,0.017857,0.071429,0.053571,0.000000,0.000000,0.00,0.00000,...,0.017857,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000


In [21]:
sg_grouped.shape

(48, 283)

### K-Means Clustering of Neighborhoods based on dataframe sg_grouped - means of venues in each Neighborhood

In [22]:
sg_grouped_clustering = sg_grouped.drop('Neighborhood', 1)

In [23]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 6

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sg_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 0, 2, 2, 2, 0, 0], dtype=int32)

In [24]:
sg_merged = df.drop(["Region", "Area(km2)",'Density(/km2)'], axis=1)
sg_merged.head()

,Neighborhood,Population,Latitude,Longitude
0,Ang Mo Kio,165710,1.369842,103.846609
1,Bedok,281300,1.323976,103.930216
2,Bishan,88490,1.351452,103.848250
3,Boon Lay,30,1.345640,103.711802
4,Bukit Batok,144410,1.349057,103.749591


In [25]:
sg_merged.insert(0,'Cluster Labels', kmeans.labels_)

In [26]:
import numpy as np
# Matplotlib and associated plotting modules

import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sg_merged['Latitude'], sg_merged['Longitude'], sg_merged['Neighborhood'], sg_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [27]:
sg_merged.head()

,Cluster Labels,Neighborhood,Population,Latitude,Longitude
0,2,Ang Mo Kio,165710,1.369842,103.846609
1,2,Bedok,281300,1.323976,103.930216
2,2,Bishan,88490,1.351452,103.848250
3,2,Boon Lay,30,1.345640,103.711802
4,0,Bukit Batok,144410,1.349057,103.749591


In [28]:
# Display the Top 10 Neighborhood in Cluster 2 sorted by Population in descending order
sg_merged.loc[sg_merged['Cluster Labels'] == 2].sort_values(by=['Population'],ascending = False).head(10)

,Cluster Labels,Neighborhood,Population,Latitude,Longitude
1,2,Bedok,281300,1.323976,103.930216
15,2,Jurong West,266720,1.339636,103.707339
39,2,Tampines,257110,1.354653,103.943571
46,2,Woodlands,252530,1.436897,103.786216
34,2,Sengkang,240640,1.390949,103.895175
13,2,Hougang,223010,1.373360,103.886091
47,2,Yishun,214940,1.428136,103.833694
0,2,Ang Mo Kio,165710,1.369842,103.846609
28,2,Punggol,161570,1.398033,103.907331
5,2,Bukit Merah,151870,1.280628,103.830591


### Investigate the number of food venues in each Neighborhood

In [29]:
sg_food_venues = sg_venues  #assign sg_venues to sg_food_venues. sg_food_venues will be modified to contain only food venues
temp = sg_venues.groupby('Venue Category').count() # groupby by Venue category for further investigation
temp

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Accessories Store,3,3,3,3,3,3
Airport,4,4,4,4,4,4
Airport Food Court,1,1,1,1,1,1
Airport Lounge,4,4,4,4,4,4
Airport Terminal,4,4,4,4,4,4
American Restaurant,14,14,14,14,14,14
Arcade,2,2,2,2,2,2
Art Gallery,8,8,8,8,8,8
Art Museum,5,5,5,5,5,5


In [30]:
temp[0:10]  #see what is in the 1st 10 rows

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Accessories Store,3,3,3,3,3,3
Airport,4,4,4,4,4,4
Airport Food Court,1,1,1,1,1,1
Airport Lounge,4,4,4,4,4,4
Airport Terminal,4,4,4,4,4,4
American Restaurant,14,14,14,14,14,14
Arcade,2,2,2,2,2,2
Art Gallery,8,8,8,8,8,8
Art Museum,5,5,5,5,5,5


In [31]:
# delete non-food venues in the 1st 10 rows
sg_food_venues = sg_food_venues.set_index('Venue Category')
drop_list = ['Accessories Store','Airport','Airport Terminal','Arcade','Art Gallery','Art Museum']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)
sg_food_venues.shape

(2965, 6)

#### Repeat the above 2 steps - systemically (but manually ... :( ...) 

In [32]:
temp[10:20]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Asian Restaurant,94,94,94,94,94,94
Athletics & Sports,2,2,2,2,2,2
Australian Restaurant,4,4,4,4,4,4
BBQ Joint,19,19,19,19,19,19
Badminton Court,1,1,1,1,1,1
Bagel Shop,2,2,2,2,2,2
Bakery,72,72,72,72,72,72
Bank,1,1,1,1,1,1
Bar,32,32,32,32,32,32


In [33]:
drop_list = ['Arts & Crafts Store','Athletics & Sports','Badminton Court','Bank']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

In [34]:
temp[20:30]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Bay,1,1,1,1,1,1
Beach,4,4,4,4,4,4
Beer Bar,4,4,4,4,4,4
Beer Garden,9,9,9,9,9,9
Belgian Restaurant,1,1,1,1,1,1
Betting Shop,1,1,1,1,1,1
Bike Trail,2,2,2,2,2,2
Bistro,17,17,17,17,17,17
Boarding House,1,1,1,1,1,1


In [35]:
drop_list = ['Basketball Court','Bay','Beach','Betting Shop','Bike Trail']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

In [36]:
temp[30:40]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Bookstore,22,22,22,22,22,22
Border Crossing,2,2,2,2,2,2
Boutique,15,15,15,15,15,15
Bowling Alley,3,3,3,3,3,3
Bowling Green,1,1,1,1,1,1
Breakfast Spot,12,12,12,12,12,12
Brewery,7,7,7,7,7,7
Bridge,3,3,3,3,3,3
Bubble Tea Shop,26,26,26,26,26,26


In [37]:
drop_list = ['Boarding House','Boat or Ferry','Bookstore','Border Crossing','Boutique','Bowling Alley','Bowling Green','Bridge']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

In [38]:
temp[40:50]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Buffet,8,8,8,8,8,8
Building,4,4,4,4,4,4
Burger Joint,13,13,13,13,13,13
Burrito Place,3,3,3,3,3,3
Bus Line,8,8,8,8,8,8
Bus Station,33,33,33,33,33,33
Bus Stop,10,10,10,10,10,10
Business Service,1,1,1,1,1,1
Butcher,1,1,1,1,1,1


In [39]:
drop_list = ['Buddhist Temple','Building','Bus Line','Bus Station','Bus Stop','Business Service']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

In [40]:
temp[50:60]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Café,146,146,146,146,146,146
Campground,1,1,1,1,1,1
Canal,1,1,1,1,1,1
Candy Store,2,2,2,2,2,2
Cantonese Restaurant,6,6,6,6,6,6
Casino,1,1,1,1,1,1
Chinese Breakfast Place,2,2,2,2,2,2
Chinese Restaurant,172,172,172,172,172,172
Chocolate Shop,1,1,1,1,1,1


In [41]:
drop_list = ['Butcher','Campground','Casino']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

In [42]:
temp[60:70]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Club House,4,4,4,4,4,4
Cocktail Bar,17,17,17,17,17,17
Coffee Shop,151,151,151,151,151,151
College Cafeteria,1,1,1,1,1,1
College Gym,2,2,2,2,2,2
College Stadium,1,1,1,1,1,1
College Theater,1,1,1,1,1,1
Comfort Food Restaurant,4,4,4,4,4,4
Concert Hall,3,3,3,3,3,3


In [43]:
drop_list = ['Clothing Store','College Gym','College Stadium','College Theater','Concert Hall']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

In [44]:
temp[70:80]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Cosmetics Shop,13,13,13,13,13,13
Cruise,5,5,5,5,5,5
Cupcake Shop,2,2,2,2,2,2
Dance Studio,4,4,4,4,4,4
Deli / Bodega,3,3,3,3,3,3
Department Store,12,12,12,12,12,12
Dessert Shop,49,49,49,49,49,49
Dim Sum Restaurant,13,13,13,13,13,13
Diner,4,4,4,4,4,4


In [45]:
drop_list = ['Convenience Store','Cosmetics Shop','Cruise','Dance Studio','Department Store']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

In [46]:
temp[80:90]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Dive Bar,1,1,1,1,1,1
Dog Run,1,1,1,1,1,1
Dumpling Restaurant,12,12,12,12,12,12
Duty-free Shop,2,2,2,2,2,2
Electronics Store,9,9,9,9,9,9
Event Space,3,3,3,3,3,3
Exhibit,4,4,4,4,4,4
Farm,5,5,5,5,5,5
Farmers Market,2,2,2,2,2,2


In [47]:
drop_list = ['Discount Store','Dog Run','Duty-free Shop','Electronics Store','Event Space','Exhibit','Farm']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

In [48]:
temp[90:100]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Filipino Restaurant,3,3,3,3,3,3
Fish & Chips Shop,1,1,1,1,1,1
Fishing Spot,2,2,2,2,2,2
Flea Market,4,4,4,4,4,4
Flower Shop,2,2,2,2,2,2
Food,1,1,1,1,1,1
Food & Drink Shop,6,6,6,6,6,6
Food Court,139,139,139,139,139,139
Food Stand,2,2,2,2,2,2


In [49]:
drop_list = ['Fishing Spot','Flea Market','Flower Shop']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

In [50]:
temp[100:110]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
French Restaurant,24,24,24,24,24,24
Fried Chicken Joint,13,13,13,13,13,13
Frozen Yogurt Shop,10,10,10,10,10,10
Fruit & Vegetable Store,2,2,2,2,2,2
Furniture / Home Store,13,13,13,13,13,13
Gaming Cafe,4,4,4,4,4,4
Garden,7,7,7,7,7,7
Gas Station,1,1,1,1,1,1
Gastropub,7,7,7,7,7,7


In [51]:
drop_list = ['Furniture / Home Store','Gaming Cafe','Garden','Gas Station']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

In [52]:
temp[110:120]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
General Entertainment,7,7,7,7,7,7
German Restaurant,2,2,2,2,2,2
Gift Shop,7,7,7,7,7,7
Golf Course,2,2,2,2,2,2
Gourmet Shop,2,2,2,2,2,2
Greek Restaurant,3,3,3,3,3,3
Grocery Store,26,26,26,26,26,26
Gun Range,3,3,3,3,3,3
Gym,28,28,28,28,28,28


In [53]:
drop_list = ['General Entertainment','Gift Shop','Grocery Store','Gun Range','Gym']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

In [54]:
temp[120:130]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Gym Pool,1,1,1,1,1,1
Hainan Restaurant,10,10,10,10,10,10
Hakka Restaurant,3,3,3,3,3,3
Halal Restaurant,8,8,8,8,8,8
Harbor / Marina,2,2,2,2,2,2
Health Food Store,1,1,1,1,1,1
High School,2,2,2,2,2,2
Historic Site,2,2,2,2,2,2
History Museum,5,5,5,5,5,5


In [55]:
drop_list = ['Gym / Fitness Center','Gym Pool','Harbor / Marina','High School','Historic Site','History Museum']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

In [56]:
temp[130:140]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Hong Kong Restaurant,3,3,3,3,3,3
Hookah Bar,2,2,2,2,2,2
Hospital,1,1,1,1,1,1
Hostel,14,14,14,14,14,14
Hot Dog Joint,1,1,1,1,1,1
Hot Spring,1,1,1,1,1,1
Hotel,78,78,78,78,78,78
Hotel Bar,6,6,6,6,6,6
Hotel Pool,1,1,1,1,1,1


In [57]:
drop_list = ['Hobby Shop','Hookah Bar','Hospital','Hostel','Hot Spring','Hotel Pool']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

In [58]:
temp[140:160]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Ice Cream Shop,29,29,29,29,29,29
Indian Restaurant,57,57,57,57,57,57
Indonesian Restaurant,8,8,8,8,8,8
Indoor Play Area,1,1,1,1,1,1
Intersection,1,1,1,1,1,1
Italian Restaurant,38,38,38,38,38,38
Japanese Curry Restaurant,2,2,2,2,2,2
Japanese Restaurant,99,99,99,99,99,99
Jazz Club,1,1,1,1,1,1


In [59]:
drop_list = ['Indoor Play Area','Intersection','Jazz Club','Jewelry Store','Karaoke Bar','Kids Store',\
             'Kitchen Supply Store','Lake','Light Rail Station','Lighthouse']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

In [60]:
temp[160:180]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Lounge,10,10,10,10,10,10
Malay Restaurant,13,13,13,13,13,13
Market,7,7,7,7,7,7
Martial Arts Dojo,2,2,2,2,2,2
Massage Studio,8,8,8,8,8,8
Mediterranean Restaurant,1,1,1,1,1,1
Metro Station,4,4,4,4,4,4
Mexican Restaurant,7,7,7,7,7,7
Middle Eastern Restaurant,4,4,4,4,4,4


In [61]:
drop_list = ['Lottery Retailer','Martial Arts Dojo','Massage Studio','Metro Station','Military Base','Miscellaneous Shop',\
             'Mobile Phone Shop','Monument / Landmark','Motorcycle Shop','Mountain','Movie Theater','Multiplex','Museum']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

In [62]:
temp[180:200]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Nail Salon,1,1,1,1,1,1
Nature Preserve,2,2,2,2,2,2
Neighborhood,2,2,2,2,2,2
Nightclub,10,10,10,10,10,10
Noodle House,63,63,63,63,63,63
Office,3,3,3,3,3,3
Other Great Outdoors,2,2,2,2,2,2
Other Nightlife,1,1,1,1,1,1
Outdoor Sculpture,1,1,1,1,1,1


In [63]:
drop_list = ['Music Venue','Nail Salon','Nature Preserve','Neighborhood','Office','Other Great Outdoors', \
             'Other Nightlife','Outdoor Sculpture','Paper / Office Supplies Store','Park','Pedestrian Plaza',\
             'Performing Arts Venue','Pet Store','Pharmacy','Pier']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

In [64]:
temp[200:220]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Planetarium,1,1,1,1,1,1
Playground,17,17,17,17,17,17
Plaza,7,7,7,7,7,7
Pool,21,21,21,21,21,21
Pool Hall,2,2,2,2,2,2
Pop-Up Shop,2,2,2,2,2,2
Portuguese Restaurant,3,3,3,3,3,3
Post Office,2,2,2,2,2,2
Pub,5,5,5,5,5,5


In [65]:
drop_list = ['Planetarium','Playground','Plaza','Pool','Pool Hall','Pop-Up Shop','Post Office','Racetrack',\
             'Recreation Center','Residential Building (Apartment / Condo)','Resort','Rest Area','River','Road','Rock Club']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

In [66]:
temp[220:240]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Salad Place,6,6,6,6,6,6
Salon / Barbershop,3,3,3,3,3,3
Sandwich Place,31,31,31,31,31,31
Satay Restaurant,1,1,1,1,1,1
Scandinavian Restaurant,1,1,1,1,1,1
Scenic Lookout,4,4,4,4,4,4
Science Museum,2,2,2,2,2,2
Sculpture Garden,2,2,2,2,2,2
Seafood Restaurant,41,41,41,41,41,41


In [67]:
drop_list = ['Salon / Barbershop','Scenic Lookout','Science Museum','Sculpture Garden','Shoe Store','Skate Park',\
             'Skating Rink','Ski Area','Soccer Field']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

In [68]:
temp[240:260]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Social Club,1,1,1,1,1,1
Soup Place,11,11,11,11,11,11
South Indian Restaurant,1,1,1,1,1,1
Spa,13,13,13,13,13,13
Spanish Restaurant,7,7,7,7,7,7
Speakeasy,4,4,4,4,4,4
Sporting Goods Shop,9,9,9,9,9,9
Sports Bar,3,3,3,3,3,3
Sports Club,1,1,1,1,1,1


In [69]:
drop_list = ['Soccer Stadium','Social Club','Spa','Sporting Goods Shop','Sports Club','Stadium','Stationery Store','Supermarket']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

In [70]:
temp[260:280]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Theme Park,1,1,1,1,1,1
Theme Park Ride / Attraction,1,1,1,1,1,1
Thrift / Vintage Store,2,2,2,2,2,2
Tourist Information Center,2,2,2,2,2,2
Toy / Game Store,2,2,2,2,2,2
Trail,11,11,11,11,11,11
Train Station,3,3,3,3,3,3
Udon Restaurant,2,2,2,2,2,2
Vegetarian / Vegan Restaurant,27,27,27,27,27,27


In [71]:
drop_list = ['Theater','Theme Park','Theme Park Ride / Attraction','Thrift / Vintage Store','Tourist Information Center','Toy / Game Store','Trail','Train Station','Video Game Store','Warehouse Store','Water Park','Waterfall','Waterfront','Wine Shop']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

In [72]:
temp[280:286]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Yoga Studio,20,20,20,20,20,20
Zoo,1,1,1,1,1,1
Zoo Exhibit,4,4,4,4,4,4


In [73]:
drop_list = ['Women\'s Store','Yoga Studio','Zoo','Zoo Exhibit']
sg_food_venues = sg_food_venues.drop(drop_list, axis=0)

#### All the non-food venues removed !!

### Further df manipulations to get the total number of food venues in each Neighborhood

In [74]:
sg_food_venues = sg_food_venues.reset_index()

# one hot encoding
sg_onehot2 = pd.get_dummies(sg_food_venues[['Venue Category']], prefix="", prefix_sep="")
sg_onehot2.head()
# add neighborhood column back to dataframe
sg_onehot2['Neighborhood'] = sg_food_venues['Neighborhood']
sg_onehot2['Neighborhood']

# move neighborhood column to the first column
#fixed_columns = [sg_onehot.columns[-1]] + list(sg_onehot.columns[:-1])
#fixed_columns
sg_onehot2 = sg_onehot2.set_index("Neighborhood")
sg_onehot2 = sg_onehot2.reset_index()
#sg_onehot = sg_onehot[fixed_columns]
sg_onehot2.head()


,Neighborhood,Airport Food Court,Airport Lounge,American Restaurant,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,...,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint
0,Ang Mo Kio,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Ang Mo Kio,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Ang Mo Kio,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Ang Mo Kio,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ang Mo Kio,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
temp2 = sg_onehot2.groupby('Neighborhood').count().reset_index()
temp2

,Neighborhood,Airport Food Court,Airport Lounge,American Restaurant,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,...,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint
0,Ang Mo Kio,62,62,62,62,62,62,62,62,62,...,62,62,62,62,62,62,62,62,62,62
1,Bedok,81,81,81,81,81,81,81,81,81,...,81,81,81,81,81,81,81,81,81,81
2,Bishan,43,43,43,43,43,43,43,43,43,...,43,43,43,43,43,43,43,43,43,43
3,Boon Lay,55,55,55,55,55,55,55,55,55,...,55,55,55,55,55,55,55,55,55,55
4,Bugis,74,74,74,74,74,74,74,74,74,...,74,74,74,74,74,74,74,74,74,74
5,Bukit Batok,27,27,27,27,27,27,27,27,27,...,27,27,27,27,27,27,27,27,27,27
6,Bukit Merah,86,86,86,86,86,86,86,86,86,...,86,86,86,86,86,86,86,86,86,86
7,Bukit Panjang,28,28,28,28,28,28,28,28,28,...,28,28,28,28,28,28,28,28,28,28
8,Bukit Timah,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9,Changi,29,29,29,29,29,29,29,29,29,...,29,29,29,29,29,29,29,29,29,29


In [76]:
#Create the df with Neighborhood and Total Food Venues
sg_food_grouped =pd.DataFrame()
sg_food_grouped['Neighborhood'] =temp2['Neighborhood']
sg_food_grouped['Total Food Venues'] = temp2['Airport Food Court']

In [77]:
#Merge food venues with latitudes and longitudes of each Neighborhood
sg_food_merged = sg_food_grouped.join(df.set_index('Neighborhood'), on='Neighborhood')
sg_food_merged = sg_food_merged.drop(['Region','Area(km2)','Density(/km2)'], axis=1)
sg_food_merged 

,Neighborhood,Total Food Venues,Population,Latitude,Longitude
0,Ang Mo Kio,62,165710,1.369842,103.846609
1,Bedok,81,281300,1.323976,103.930216
2,Bishan,43,88490,1.351452,103.848250
3,Boon Lay,55,30,1.345640,103.711802
4,Bugis,74,2510,1.299476,103.855089
5,Bukit Batok,27,144410,1.349057,103.749591
6,Bukit Merah,86,151870,1.280628,103.830591
7,Bukit Panjang,28,140820,1.377921,103.771866
8,Bukit Timah,1,77280,1.354690,103.776372
9,Changi,29,2080,1.352516,103.987007


In [78]:
# Visualize Food Venues
map_sg_food = folium.Map(location=[latitude, longitude], zoom_start=11)

for i in range(0,len(sg_food_merged)):
    
    folium.Circle(
      location=[sg_food_merged.iloc[i]['Latitude'], sg_food_merged.iloc[i]['Longitude']],
      popup=(sg_food_merged.iloc[i]['Neighborhood']+'\n'+str(sg_food_merged.iloc[i]['Total Food Venues'])),
      radius=sg_food_merged.iloc[i]['Total Food Venues'] / .1,
      color='purple',
      fill=True,
      fill_color='purple'
   ).add_to(map_sg_food)
map_sg_food

In [79]:
sg_food_merged.head()

,Neighborhood,Total Food Venues,Population,Latitude,Longitude
0,Ang Mo Kio,62,165710,1.369842,103.846609
1,Bedok,81,281300,1.323976,103.930216
2,Bishan,43,88490,1.351452,103.848250
3,Boon Lay,55,30,1.345640,103.711802
4,Bugis,74,2510,1.299476,103.855089


In [80]:
# Calculate the ratio of Population to Food Venues in each neighborhood and assign the result to a new column
sg_food_merged['Pop_to_Food_Ratio'] = (sg_food_merged['Population']/sg_food_merged['Total Food Venues']).astype(int)

In [81]:
# Display the Top 10 Neighborhoods sorted by Population in descending order
sg_food_merged.sort_values(by=['Population'],ascending = False).head(10)

,Neighborhood,Total Food Venues,Population,Latitude,Longitude,Pop_to_Food_Ratio
1,Bedok,81,281300,1.323976,103.930216,3472
17,Jurong West,50,266720,1.339636,103.707339,5334
40,Tampines,51,257110,1.354653,103.943571,5041
44,Woodlands,43,252530,1.436897,103.786216,5872
35,Sengkang,31,240640,1.390949,103.895175,7762
14,Hougang,42,223010,1.373360,103.886091,5309
45,Yishun,34,214940,1.428136,103.833694,6321
10,Choa Chu Kang,28,187510,1.389260,103.743728,6696
0,Ang Mo Kio,62,165710,1.369842,103.846609,2672
29,Punggol,15,161570,1.398033,103.907331,10771


In [82]:
sg_food_merged.sort_values(by=['Pop_to_Food_Ratio'],ascending = False).head(10)

,Neighborhood,Total Food Venues,Population,Latitude,Longitude,Pop_to_Food_Ratio
8,Bukit Timah,1,77280,1.354690,103.776372,77280
29,Punggol,15,161570,1.398033,103.907331,10771
35,Sengkang,31,240640,1.390949,103.895175,7762
10,Choa Chu Kang,28,187510,1.389260,103.743728,6696
45,Yishun,34,214940,1.428136,103.833694,6321
44,Woodlands,43,252530,1.436897,103.786216,5872
5,Bukit Batok,27,144410,1.349057,103.749591,5348
17,Jurong West,50,266720,1.339636,103.707339,5334
14,Hougang,42,223010,1.373360,103.886091,5309
40,Tampines,51,257110,1.354653,103.943571,5041
